# OAO Reader - "OnAndOn" format of the Motion GPS

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys
import time

import numpy as np

import unittest

from base_reader import BaseReader

## Main Class

In [2]:
class OaoReader(BaseReader):
    '''OAO file - "OnAndOn" format of the Motion GPS'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)


    def load(self, ignoreChecksums=False):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            self.buffer = f.read()
            
            # The memoryview will be used to access the buffer without intermediate copying
            self.bufferView = memoryview(self.buffer)

            if len(self.buffer) > 0:
                self.readData(ignoreChecksums=ignoreChecksums)


    def readData(self, ignoreChecksums=False):
        '''Read GNSS frames into memory'''

        fileSize = len(self.buffer)

        # Lean buffer will only contain the geodetic navigation data, minus start / end tokens and checksums
        leanBuffer = bytearray()

        while self.bufferPtr < fileSize:
            mode, payload = self.readFrame(ignoreChecksums=ignoreChecksums)
            
            if mode in (0x0ad4, 0x0ad5):
                # Use += because it is faster than extend - see https://stackoverflow.com/q/40004517
                leanBuffer += payload
                self.numRecords += 1
                       
        if self.bufferPtr != fileSize:
            raise ValueError('File appears to be truncated')

        dtype = self.getDatatype()
        
        self.rawData = np.frombuffer(leanBuffer, dtype=dtype, count=self.numRecords)
        
        self.consumeRawData()


    def readFrame(self, ignoreChecksums=False):
        '''Read OAO frame from buffer'''

        offset = self.bufferPtr

        mode = self.readUnsigned16LE()
        expectedChecksum = self.readUnsigned16LE()

        if mode == 0x0ad0:
            length = 512
        elif mode == 0x0ad1:
            length = 12
        elif mode == 0x0ad2 or mode == 0x0ad3:
            length = 34
        elif mode == 0x0ad4 or mode == 0x0ad5:
            length = 52
        else:
            raise ValueError('Unexpected mode 0x{:x} - offset 0x{:08x}'.format(mode, offset))

        # Main frame content
        payload = self.readBytes(length - 4)

        # Evaluate checksum
        if ignoreChecksums == False:
            actualChecksum = self.calculateChecksum(mode, payload)
            if actualChecksum != expectedChecksum:
                raise ValueError('Checksum difference - {} vs {}'.format(actualChecksum, expectedChecksum))

        return mode, payload


    def calculateChecksum(self, mode, payload):
        '''Calculate Fletcher's checksum for the OAO frame'''

        checksum_a = checksum_b = 0
        
        # Sum the two first bytes
        for i in range(2):
            checksum_a += mode
            checksum_b += checksum_a
            mode >>= 8

        # Sum the payload bytes
        for byte in payload:
            checksum_a += byte
            checksum_b += checksum_a

        # Bit masking has been deferred until the end
        return (checksum_b & 0xff) << 8 | checksum_a & 0xff


    def getDatatype(self):
        '''Get the datatype of the GNSS frames'''

        # Standard GNSS frame (mode = 0x0ad4 or 0x0ad5) - 48 bytes excluding the mode and checksum
        dtype = [
            ('latitude', '<i4'),
            ('longitude', '<i4'),
            ('altitude_msl', '<i4'),
            ('speed', '<u4'),
            ('course', '<u4'),
            ('utc_time', '<u8'),
            ('fix', 'u1'),
            ('satellites', 'u1'),
            ('speed_accuracy', '<u4'),
            ('horizontal_accuracy', '<u4'),
            ('vertical_accuracy', '<u4'),
            ('heading_accuracy', '<u4'),
            ('hdop', '<u2')
        ]

        return np.dtype(dtype)


    def consumeRawData(self):
        '''Consume data - convert to standard data structure'''

        # Convert latitude and longitude to decimal
        self.data['lat'] = np.divide(self.rawData['latitude'], 10000000, dtype='float64')
        self.data['lon'] = np.divide(self.rawData['longitude'], 10000000, dtype='float64')

        # Convert elevation from cm to m
        self.data['ele'] = np.divide(self.rawData['altitude_msl'], 1000, dtype='float64')

        # Convert SOG from mm/s to m/s
        self.data['sog'] = np.divide(self.rawData['speed'], 1000, dtype='float32')

        # Convert COG from integer to decimal
        self.data['cog'] = np.round(np.divide(self.rawData['course'], 100000, dtype='float32'), 3)

        # Convert timestamp from ms to s
        self.data['ts'] = np.divide(self.rawData['utc_time'], 1000, dtype='float64')

        # Copy fix and satellites
        self.data['fix'] = self.rawData['fix']
        self.data['sat'] = self.rawData['satellites']

        # Convert sAcc from mm/s to m/s
        self.data['sacc'] = np.divide(self.rawData['speed_accuracy'], 1000, dtype='float32')

        # Convert hAcc and vAcc from mm to m
        self.data['hacc'] = np.divide(self.rawData['horizontal_accuracy'], 1000, dtype='float32')
        self.data['vacc'] = np.divide(self.rawData['vertical_accuracy'], 1000, dtype='float32')

        # Convert cAcc to degrees
        self.data['cacc'] = np.round(np.divide(self.rawData['heading_accuracy'], 100000, dtype='float32'), 3)

        # Convert HDOP from integer to decimal
        self.data['hdop'] = np.divide(self.rawData['hdop'], 100, dtype='float32')

## Unit Tests

In [3]:
class TestMiniData(unittest.TestCase):
    '''Class to test Motion Mini data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(miniReader.numRecords, 47980)

        for fieldName in miniReader.data:
            self.assertEqual(miniReader.data[fieldName].size, 47980)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(miniReader.data['hdop'].min(), np.float32(0.62))
        self.assertEqual(miniReader.data['hdop'].max(), np.float32(99.99))


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(miniReader.data['sat'].min(), 0)
        self.assertEqual(miniReader.data['sat'].max(), 18)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(miniReader.data['ts'].min(), np.float64(1649672228.0))
        self.assertEqual(miniReader.data['ts'].max(), np.float64(1649677025.9))


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(miniReader.data['lat'].min(), np.float64(50.5710305))
        self.assertEqual(miniReader.data['lat'].max(), np.float64(50.5833157))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(miniReader.data['lon'].min(), np.float64(-2.462027))
        self.assertEqual(miniReader.data['lon'].max(), np.float64(-2.4563075))


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(miniReader.data['ele'].min(), np.float64(-3.169))
        self.assertEqual(miniReader.data['ele'].max(), np.float64(13.548))


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(miniReader.data['sog'].min(), np.float32(0.000))
        self.assertEqual(miniReader.data['sog'].max(), np.float32(17.142))


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(miniReader.data['cog'].min(), np.float32(0.001))
        self.assertEqual(miniReader.data['cog'].max(), np.float32(359.996))


    def testFix(self):
        '''Test the fix is as expected'''

        self.assertEqual(miniReader.data['fix'].min(), 0)
        self.assertEqual(miniReader.data['fix'].max(), 3)


    def testSacc(self):
        '''Test the speed accuracy is as expected'''

        self.assertEqual(miniReader.data['sacc'].min(), np.float32(0.092))
        self.assertEqual(miniReader.data['sacc'].max(), np.float32(2.118))


    def testHacc(self):
        '''Test the horizontal accuracy is as expected'''

        self.assertEqual(miniReader.data['hacc'].min(), np.float32(0.395))
        self.assertEqual(miniReader.data['hacc'].max(), np.float32(12.051))


    def testVacc(self):
        '''Test the vertical accuracy is as expected'''

        self.assertEqual(miniReader.data['vacc'].min(), np.float32(0.524))
        self.assertEqual(miniReader.data['vacc'].max(), np.float32(11.574))


    def testCacc(self):
        '''Test the course accuracy is as expected'''

        self.assertEqual(miniReader.data['cacc'].min(), np.float32(0.401))
        self.assertEqual(miniReader.data['cacc'].max(), np.float32(45.016))

In [4]:
class TestEspData(unittest.TestCase):
    '''Class to test ESP-GPS data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(espReader.numRecords, 43444)

        for fieldName in espReader.data:
            self.assertEqual(espReader.data[fieldName].size, 43444)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        # Note: HDOP in this file contains NAV-PVT.pDOP instead of NAV-DOP.hDOP
        self.assertEqual(espReader.data['hdop'].min(), np.float32(0.96))
        self.assertEqual(espReader.data['hdop'].max(), np.float32(1.88))


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(espReader.data['sat'].min(), 9)
        self.assertEqual(espReader.data['sat'].max(), 18)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(espReader.data['ts'].min(), np.float64(1640864825.3))
        self.assertEqual(espReader.data['ts'].max(), np.float64(1640869169.6))


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(espReader.data['lat'].min(), np.float64(51.6993591))
        self.assertEqual(espReader.data['lat'].max(), np.float64(51.7087559))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(espReader.data['lon'].min(), np.float64(4.0860906))
        self.assertEqual(espReader.data['lon'].max(), np.float64(4.0890658))


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(espReader.data['ele'].min(), np.float64(-3.159))
        self.assertEqual(espReader.data['ele'].max(), np.float64(11.978))


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(espReader.data['sog'].min(), np.float32(0.000))
        self.assertEqual(espReader.data['sog'].max(), np.float32(18.169))


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(espReader.data['cog'].min(), np.float32(0.011))
        self.assertEqual(espReader.data['cog'].max(), np.float32(359.995))


    def testFix(self):
        '''Test the fix is as expected'''

        self.assertEqual(espReader.data['fix'].min(), 3)
        self.assertEqual(espReader.data['fix'].max(), 3)


    def testSacc(self):
        '''Test the speed accuracy is as expected'''

        self.assertEqual(espReader.data['sacc'].min(), np.float32(0.051))
        self.assertEqual(espReader.data['sacc'].max(), np.float32(0.771))


    def testHacc(self):
        '''Test the horizontal accuracy is as expected'''

        self.assertEqual(espReader.data['hacc'].min(), np.float32(0.169))
        self.assertEqual(espReader.data['hacc'].max(), np.float32(2.508))


    def testVacc(self):
        '''Test the vertical accuracy is as expected'''

        self.assertEqual(espReader.data['vacc'].min(), np.float32(0.207))
        self.assertEqual(espReader.data['vacc'].max(), np.float32(2.582))


    def testCacc(self):
        '''Test the course accuracy is as expected'''

        self.assertEqual(espReader.data['cacc'].min(), np.float32(0.239))
        self.assertEqual(espReader.data['cacc'].max(), np.float32(44.999))

In [5]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220411', 'Motion_Mini_10Hz_0470_2022-04-11-1117.oao')
    miniReader = OaoReader(filename)

    filename = os.path.join(projdir, 'sessions', '20211230-esp', 'Head_L_7C9EBDFAF5C8_007.oao')
    espReader = OaoReader(filename)

    pc1 = time.perf_counter()
    miniReader.load()
    espReader.load()
    pc2 = time.perf_counter()

    print("\nOAO files loaded in in %0.2f seconds" % (pc2 - pc1))


OAO files loaded in in 0.52 seconds


In [6]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

............................
----------------------------------------------------------------------
Ran 28 tests in 0.032s

OK
